# Finding Similar Movies
 as 'Cinderella'
## Data Source [MovieLens](https://grouplens.org/datasets/movielens/100k/)

Find the similar rating movies by rating:
Build a matrix of user rated
Find similarity scores among users
Find similar users
Recommend stuff they viewed

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)
print(ratings.head(2))

   movie_id             title  user_id  rating
0         1  Toy Story (1995)      308       4
1         1  Toy Story (1995)      287       5


In [2]:
# pivot_table function can build a user / movie rating matrix. N/A means no rating data
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head(2)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN


In [3]:
# extract users who rated 'Cinderella (1950)':
starWarsRatings = movieRatings['Cinderella (1950)']
starWarsRatings.head(3)

user_id
0   NaN
1   NaN
2   NaN
Name: Cinderella (1950), dtype: float64

In [4]:
#Use corrwith function to compute the pairwise correlation of 'Cinderella' vector of user rating with every other movie
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
similarMovies.sort_values(ascending=False)

D:\Program\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
D:\Program\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
Kansas City (1996)                             1.0
Cinderella (1950)                              1.0
One Night Stand (1997)                         1.0
Loch Ness (1995)                               1.0
Indian Summer (1996)                           1.0
                                              ... 
Tom & Viv (1994)                              -1.0
Fire Down Below (1997)                        -1.0
Solo (1996)                                   -1.0
Two or Three Things I Know About Her (1966)   -1.0
Clean Slate (Coup de Torchon) (1981)          -1.0
Length: 1212, dtype: float64

In [5]:
# get rid of movies that were watched by fewer than 100 people few people 
import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
popularMovies = movieStats['rating']['size'] >= 200
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:5]

rating          
                                   size      mean
title                                            
Schindler's List (1993)             298  4.466443
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230
Rear Window (1954)                  209  4.387560
Usual Suspects, The (1995)          267  4.385768

In [6]:
# join this data with similar movies to Star Wars:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))
df.sort_values(['similarity'], ascending=False)[:5]

D:\Program\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similarity
title,,,
Mission: Impossible (1996),344,3.313953,0.506632
Aladdin (1992),219,3.812785,0.481009
Beauty and the Beast (1991),202,3.792079,0.468520
It's a Wonderful Life (1946),231,4.121212,0.462092
"Lion King, The (1994)",220,3.781818,0.448823
